In [83]:
import pandas as pd
import numpy as np

In [84]:
base_train = pd.read_csv('deep/titanic/Titanic-Kaggle/csv/train.csv')
base_test = pd.read_csv('deep/titanic/Titanic-Kaggle/csv/test.csv')

In [85]:
base_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [86]:
base_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [87]:
base_train = base_train.drop(['PassengerId', 'Name', 'Ticket'], axis = 1)
base_test = base_test.drop(['PassengerId', 'Name', 'Ticket'], axis = 1)

In [88]:
print('Treino')
base_train.info()
print('Teste')
base_train.info()

Treino
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB
Teste
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch    

In [89]:
#analisado se ha dados nulos na base de treino
base_train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [49]:
#tratando dados nulos na base de treino
media_age = int(base_train.Age.mean())
base_train.update(base_train['Age'].fillna(float(media_age)))
base_train.update(base_train['Cabin'].fillna('Nao Cadastrada'))
base_train.update(base_train['Embarked'].fillna('C'))

In [93]:
#analisado se ha dados nulos na base de teste
base_test.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [92]:
#tratando dados nulos da base de teste
media_age_teste = int(base_test.Age.mean())
base_test.update(base_test['Age'].fillna(float(media_age_teste)))
media_fare_teste = int(base_test.Fare.mean())
base_test.update(base_test['Fare'].fillna((media_fare_teste)))
base_test.update(base_test['Cabin'].fillna('Nao Cadastrada'))

In [59]:
x_train = base_train.drop('Survived', axis = 1)

In [60]:
y_train = base_train.Survived

In [62]:
x_train.shape

(891, 8)

In [64]:
y_train.shape

(891,)

In [65]:
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [66]:
#tratando colunas numericas
f_pclass = tf.feature_column.numeric_column(key='Pclass')
f_age = tf.feature_column.numeric_column(key='Age')
f_sibSp = tf.feature_column.numeric_column(key='SibSp')
f_parch = tf.feature_column.numeric_column(key='Parch')
f_fare = tf.feature_column.numeric_column(key='Fare')

In [67]:
#tratando colunas categoricas
f_sex = tf.feature_column.categorical_column_with_vocabulary_list('Sex', ['female', 'male'])
f_cabin = tf.feature_column.categorical_column_with_hash_bucket('Cabin', hash_bucket_size = 100)
f_embarked = tf.feature_column.categorical_column_with_hash_bucket('Embarked', hash_bucket_size = 4)

In [68]:
colunas = [f_pclass, f_age, f_sibSp, f_parch,
        f_fare, f_cabin, f_embarked]

In [69]:
estimator = tf.estimator.LinearClassifier(feature_columns=colunas)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmppcc0j8kk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [94]:
from sklearn.model_selection import train_test_split 
x_treinamento, x_test, y_treinamento, y_test = train_test_split(x_train, y_train, test_size=0.2)

In [95]:
x_test.shape

(179, 8)

In [96]:
y_train.shape

(891,)

In [97]:
y_test.head()

843    0
29     0
866    1
580    1
273    0
Name: Survived, dtype: int64

In [98]:
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = x_treinamento, y = y_treinamento, num_epochs = None, shuffle = True, target_column = 'target')

funcao_teste = tf.estimator.inputs.pandas_input_fn(x = x_test, y = y_test, num_epochs = 1, shuffle = False)


In [101]:
estimator.train(input_fn=funcao_treinamento, steps = 400)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmppcc0j8kk/model.ckpt-800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 800...
INFO:tensorflow:Saving checkpoints for 800 into /tmp/tmppcc0j8kk/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 800...
INFO:tensorflow:loss = 68.662766, step = 801
INFO:tensorflow:global_step/sec: 423.458
INFO:tensorflow:loss = 72.030205, step = 901 (0.238 sec)
INFO:tensorflow:global_step/sec: 455.06
INFO:tensorflow:loss = 71.97081, step = 1001 (0.221 sec)
INFO:tensorflow:global_step/sec: 518.604
INFO:tensorflow:loss = 69.655014, step = 1101 (0.191 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 1200...
INFO:tensorflow:Saving checkpoints for 120

In [102]:
estimator.evaluate(input_fn=funcao_teste)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-06-09T17:33:34Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmppcc0j8kk/model.ckpt-1200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.44420s
INFO:tensorflow:Finished evaluation at 2020-06-09-17:33:34
INFO:tensorflow:Saving dict for global step 1200: accuracy = 0.75418997, accuracy_baseline = 0.6368715, auc = 0.73886645, auc_precision_recall = 0.6928971, average_loss = 0.58085054, global_step = 1200, label/mean = 0.36312848, loss = 51.986122, precision = 0.8, prediction/mean = 0.39969358, recall = 0.43076923
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1200: /tmp/tmppcc0j8kk/model.ckpt-1200


{'accuracy': 0.75418997,
 'accuracy_baseline': 0.6368715,
 'auc': 0.73886645,
 'auc_precision_recall': 0.6928971,
 'average_loss': 0.58085054,
 'label/mean': 0.36312848,
 'loss': 51.986122,
 'precision': 0.8,
 'prediction/mean': 0.39969358,
 'recall': 0.43076923,
 'global_step': 1200}